# contents(), singlesection(), refInSection() actsInSection(), references()

In [13]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from typing import Optional, Any, List, Union, TypeVar, Type, cast, Callable
import re
from statistics import mean 


# for each subsection, consider adding into the current memory object
# the leaves text/structure

class Part:
    def __init__(self, Number, Title, Chapters):
        self.Number = Number
        self.Title = Title
        self.Chapters = Chapters

class Chapter:
    def __init__(self, Number, Title, Sections):
        self.Number = Number
        self.Title = Title
        self.Sections= Sections

class SubSection:
    Title: str
    Number: int
    Url: str
    def __init__(self, Number, Title, Url):
        self.Number = Number
        self.Title = Title
        self.Url = Url

class Section:
    Title: str
    SubSections: Optional[List[SubSection]]
    def __init__(self, Title, SubSections):
        self.Title = Title
        self.SubSections = SubSections        

class SuperSections:
    partNumber: int
    partTitle: str
    chapterNumber: int
    chapterTitle: str
    crossHeading: str
    section: Section
    subSectionsNum: int
    paragraphsNum: int
    subParagraphsNum: int
    linesNum: int
    ingoing: dict
    outgoing: dict        
    def __init__(self, section, subSections, paragraphs, subParagraphs, lines, ingoing, outgoing, partNumber, partTitle, chapterNumber, chapterTitle, crossHeading):
        self.partNumber= partNumber
        self.partTitle= partTitle   
        self.chapterNumber=chapterNumber
        self.chapterTitle=chapterTitle
        self.crossHeading=crossHeading
        self.section = section
        self.subSectionsNum = subSections        
        self.paragraphsNum = paragraphs        
        self.subParagraphsNum = subParagraphs        
        self.linesNum = lines  
        self.ingoing = ingoing        
        self.outgoing = outgoing

def contents(url, type=1):
    urls = []
    ids = []
    txt = ''
    parts = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, features="xml")
    for part in  soup.select('Body > Part'):
        txt = txt + " <p> <b>"+part.find('Number').text+" "+part.find('Title').text+"</b></p>"
        chapters=[]
        if(len(part.find_all('Chapter'))>0):
            for chapter in part.find_all('Chapter'):
              # keep chapter
                if(len(chapter.find_all('Pblock'))>0):
                    subsections=[]
                    sections=[]
                    for result in chapter.find_all('Pblock'):
                        subsections=[]
                        txt = txt + " <p> " +result.find('Title').text+"</p>"
                        for first in result.select('Pblock > P1group'):
                            ids.append(first.select('Pnumber')[0].text)
                            txt = txt + " <p> "+first.select('Pnumber')[0].text+". "+first.find('Title').text+"</p>"
                            p1=first.findAll("P1")
                            if p1[0].has_attr('DocumentURI'):
                                urls.append(p1[0]['DocumentURI']+"/data.xml")
                                subsections.append(SubSection(first.select('Pnumber')[0].text, first.find('Title').text, p1[0]['DocumentURI']+"/data.xml"))
                        sections.append(Section(result.find('Title').text, subsections))
                else:
                    subsections=[]
                    sections=[]
                    for first in chapter.select('P1group'):
                            txt = txt + " <p> " +first.select('Pnumber')[0].text+". "+first.find('Title').text+"</p>"
                            ids.append(first.select('Pnumber')[0].text)
                            p1=first.findAll("P1")
                            if p1[0].has_attr('DocumentURI'):
                                urls.append(p1[0]['DocumentURI']+"/data.xml")
                                subsections.append(SubSection(first.select('Pnumber')[0].text, first.find('Title').text, p1[0]['DocumentURI']+"/data.xml"))
                    sections.append(Section(chapter.find('Title').text, subsections))
                chapters.append(Chapter(chapter.find('Number').text, chapter.find('Title').text, sections))                
                
        else:
                sections=[]
                for result in part.find_all('Pblock'):
                    txt = txt + " <p> " +result.find('Title').text+"</p>"
                    # code replication - improve
                    subsections=[]
                    for first in result.select('Pblock > P1group'):
                            if(len(first.findAll('P1'))>0):
                                txt = txt + " <p> " +first.select('Pnumber')[0].text+". "+first.find('Title').text+"</p>"
                                ids.append(first.select('Pnumber')[0].text)
                                p1=first.findAll("P1")
                                if p1[0].has_attr('DocumentURI'):
                                    urls.append(p1[0]['DocumentURI']+"/data.xml")
                                    subsections.append(SubSection(" "+first.select('Pnumber')[0].text, first.find('Title').text, p1[0]['DocumentURI']+"/data.xml"))
                    sections.append(Section(result.find('Title').text, subsections))
                chapters.append(Chapter("", "", sections))

        parts.append(Part(part.find('Number').text, part.find('Title').text, chapters))

    if type == 1:
     return parts
    else:
     return parts


year=2004
number=34

parts=contents('https://www.legislation.gov.uk/ukpga/'+str(year)+'/'+str(number)+'/data.xml')


In [14]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from typing import Optional, Any, List, Union, TypeVar, Type, cast, Callable

class Line:
    Title: str
    Letter: str
    def __init__(self, Letter, Title):
        self.Letter = Letter
        self.Title = Title   

class subParagraph:
    Title: str
    Number: str
    Lines: Optional[List[Line]]
    def __init__(self,  Number, Title,Lines):
        self.Title = Title   
        self.Number = Number
        self.Lines = Lines
        
class Paragraph:
    Title: str
    Number: int
    subParagraphs: Optional[List[subParagraph]]
    def __init__(self, Number, Title, subParagraphs):
        self.Number = Number
        self.Title = Title
        self.subParagraphs = subParagraphs

class subSections:
    Title: str
    Number: int
    Paragraphs: Optional[List[Paragraph]]
    def __init__(self, Title, Number, Paragraphs):
        self.Title = Title
        self.Number = Number
        self.Paragraphs = Paragraphs

class Sections:
    Title: str
    Number: int
    subSections: Optional[List[subSections]]
    def __init__(self, Title, Number, subSections):
        self.Title = Title
        self.Number = Number
        self.subSections = subSections

def singlesection(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "xml")
        p5condition = False
        lists = False
        txt = ''
        pattern='Act '
        parts=[]
        partTitle=None
        itemNumber=None
        for part in soup.find_all('Part'):
            # Chapter 4, 6
            if (len(part.select('Pblock > P1group')) == 0):
                path = 'P1group'
            else:
                path = 'Pblock > P1group'
            for first in part.select(path):
                txt = txt + "<p>" + first.select('Pnumber')[0].text + ". " + first.find('Title').text+"</p>"
                titleSection= first.find('Title').text
                numberSection= first.select('Pnumber')[0].text
                p1 = first.findAll("P1")
                if p1[0].has_attr('DocumentURI'):
                    txt = txt + "<p>" + p1[0]['DocumentURI']+"</p>"
                    if first.select('P1 > P1para >Text'):
                        txt = first.select('P1 > P1para >Text')[0].text
                    p1paras = first.select('P1para');
                    blocks = first.select('BlockAmendment > P1group > P1 > P1para');
                    topPara1s = len(p1paras) - len(blocks)
                    for p1para in p1paras[:topPara1s]:
                        if (p1para.find('P2') == None):
                            elements = []
                        elif (len(p1para.select('P5'))):
                            p5condition = True
                            elements = p1para.find_all_next()
                        else:
                            elements = p1para.find_all_next()
                        
                        # small sections, say 77, 99
                        if(len(elements)==0):
                            subParagraphs=[];
                            subParts=[];
                            
                            textArray = p1para.select('P3 > P3para >Text')  
                            for p3para in textArray:
                                txt = " <p> " + p3para.text+ " </p>"
                                subParts.append(Paragraph("", p3para.text,[]))

                            for listItem in p1para.select('UnorderedList >ListItem'):
                                txt = listItem.find('Para').text
                                subParagraphs.append(subParagraph('',  listItem.find('Para').text, []));
                            if(len(p1para.select('P1para > Text'))>1):
                                subParagraphs.append(subParagraph('', p1para.select('P1para > Text')[1].text, []));
                                
                            subParts.append(Paragraph("","",subParagraphs))            
                            parts.append(subSections('', txt,subParts))
                            return Sections(titleSection, numberSection, parts)
                            
                        for element in elements:
                            subParagraphs=[];
                            subParts=[];
                            if (element.name == 'P2'):
                                if (element.has_attr('DocumentURI')):
                                    txt = txt + "  <p> " + element.select('Pnumber')[0].text+" </p>"
                                    txt = txt + "  <p> " + element.select('P2para > Text')[0].text+" </p>"
                                    partNumber = element.select('Pnumber')[0].text;
                                    partTitle  = element.select('P2para > Text')[0].text
                                    p2 = element;
                                    #legislation amendments
                                    if (len(p2.select('P2 > P2para > BlockAmendment > P1group')) != 0 and not p5condition):
                                        for block in p2.select('P2 > P2para > BlockAmendment > P1group'):
                                            textIndex = 0
                                            legislationSubsectionNum=block.select('P1 > Pnumber')[0].text;
                                            legislationSubsectionTitle=block.select('Title')[0].text;
                                            items=[]
                                            for p1 in block.select('P1 > P1para > P2'):
                                                subsectionTitle=p1.select('P2para > Text')[0].text
                                                subsectionNumber=(p1.select('Pnumber')[0].text)
                                                # 185
                                                p3s = p1.select('P2 > P2para > P3')
                                                final_text = ""
                                                for p3 in p3s:
                                                    subPartNumber=p3.select('Pnumber')[0].text;
                                                    subPartTitle=p3.find('Text').text;
                                                    final_text = p3.find_next_sibling('Text')
                                                    items.append(Line(p3.select('Pnumber')[0].text, p3.find('Text').text))
                                                    
                                                    for p4 in p3.select('P3 > P3para > P4'):
                                                        array = p4.select('P2 > P2para > Text')
                                                        for p2para in array:
                                                            txt = txt + " <p> " + p2para.text+" </p>"
#                                                             items.append(Line('', p2para.text))
#                                                         subParagraphs.append(subParagraph(p4.select('Pnumber')[0].text, p4.find('Text').text, items))
                                            
                                                if (type(final_text) is str):  # 185, 179
                                                    txt = txt + "  <p> " +final_text+" </p>"
#                                                 subParagraphs.appends(subParagraph('', final_text, []))


                                                elif (hasattr(final_text, 'text')):
                                                    txt = txt + "  <p> " + final_text.text+" </p>"
                                                    items.append(Line('', final_text.text))
#                                                 if(len(items)>0):
#                                                     subParagraphs.append(subParagraph('','',items))
                                                subParts.append(Paragraph(subsectionTitle,subsectionNumber,subParagraphs))

  
    # Section 206 5, last two lines
#                                                 array = p1.select('P2 > P2para > Text')
#                                                 del array[0]
#                                                 for p2para in array:
#                                                         print("Hello There"+str(p2para))
                
#                                             parts.append(subSections(legislationSubsectionTitle, legislationSubsectionNum,subParts))

                                    elif (len(p2.select('P5')) != 0):
                                        txt = txt + "<p>" + p5(p2) + " </p>"
                                    elif (len(p2.select(
                                            'P2 > P2para > BlockAmendment')) != 0):  # 211( Text -> P3), #5 (P2 -> P2Para),#185
                                        for block in p2.select('P2 > P2para > BlockAmendment'):
                                            textArray = block.select('Text')
                                            rootItems=[]
                                            if (len(textArray) > 0):    
                                                # validate this step
                                                for p3 in block.select('BlockAmendment > P3'):
                                                    txt = txt + " <p> " + p3.select('Pnumber')[0].text + " " + p3.find('Text').text+" </p>"
                                                    
                                                for p2 in block.select('BlockAmendment > P2'):
                                                    text_counter=0
                                                    txt = txt + "  <p> " + p2.select('Pnumber')[0].text + " " + p2.find('Text').text+" </p>"
                                                    subPartNumber=p2.select('Pnumber')[0].text;
                                                    subPartTitle=p2.find('Text').text;
                                                    text_counter=text_counter+1
                                                    items=[]
                                                    for p3 in p2.select('P2para > P3'):
                                                        txt = txt + " <p> " + p3.select('Pnumber')[0].text + " " + p3.find('Text').text+" </p>"
                                                        if(p3.select('Pnumber')[0].text != None):
                                                            items.append(Line(p3.select('Pnumber')[0].text, p3.find('Text').text))
                                                        text_counter=text_counter+1                                                        
                                                    ## for loop ?? - an example for 193
                                                    if(text_counter < len(p2.select('Text'))):
                                                        txt = txt + " <p> " + p2.select('Text')[text_counter].text + " </p>"
                                                        if(p2.select('Text')[text_counter].text != None):
                                                            items.append(Line(" ", p2.select('Text')[text_counter].text))
                                                    if(len(items)>0):
                                                        subParts.append(Paragraph(subPartNumber,subPartTitle,[subParagraph(' ', '',items)]))
                                                    else:
                                                        subParts.append(Paragraph(subPartNumber,subPartTitle,[]))

                                    elif (len(p2.select('P2 > P2para > P3')) != 0):
                                        if (p2.has_attr('DocumentURI')):
                                            for p3 in p2.select('P2 > P2para > P3'):
                                                txt = txt + " " + p3.select('Pnumber')[0].text + " " + p3.find('Text').text + " </p>"
                                                subPartNumber = p3.select('Pnumber')[0].text;
                                                subPartTitle = p3.find('Text').text;  
                                                subParagraphs=[];
                                                items=[]
                                                for p4 in p3.select('P3 > P3para > P4'):
                                                    # PNUMBER and text display
                                                    txt =" <p> " + p4.select('Pnumber')[0].text + " " + p4.find('Text').text+ " </p>"
                                                    subParagraphs.append(subParagraph(p4.select('Pnumber')[0].text, p4.find('Text').text,[]))
                                                subParts.append(Paragraph(subPartNumber, subPartTitle,subParagraphs))

                                            textArray = p3.select('P3 > P3para >Text')  # 5
                                            del textArray[0]
                                            for p3para in textArray:
                                                txt = txt + " <p> " + p3para.text+ " </p>"
                                                if(p3para.text != None):
                                                    subParts.append(Paragraph("", p3para.text,[]))

                                            array = p2.select('P2 > P2para > Text')
                                            del array[0]
                                            for p2para in array:
                                                if(p2para.text != None):
                                                    subParts.append(Paragraph("", p2para.text,[]))

                                                
                                    
                                    if (len(p2.find_all('UnorderedList')) > 0):
                                        lists = True
                                    rootItems=[]
                                    for unorderedlist in p2.find_all('UnorderedList'):
                                        for listItem in unorderedlist.select('UnorderedList >ListItem'):
                                            txt = txt + "  && <p> " + listItem.find('Para').text+ " </p>"
                                            titleSubPart = listItem.find('Para').text;
                                            items=[]
                                            for orderList in listItem.find_all('OrderedList'):
                                                for listItem1 in orderList.find_all('ListItem'):
                                                    if (listItem1.has_key('NumberOverride')):
                                                        itemNumber=listItem1['NumberOverride'];
                                                    itemTitle=listItem1.find('Para').text;
                                                    items.append(Line(itemNumber, itemTitle))
                                            rootItems.append(subParagraph('',titleSubPart,items))
                                    if(len(rootItems)>0):
                                         subParts.append(Paragraph("","",rootItems))
                                    if(partTitle!=None): 
#                                         if(('substitute—' not  in partTitle) and ('insert—' not  in partTitle)) :
                                            parts.append(subSections(partTitle, partNumber,subParts))

                        
                            elif (element.name == 'UnorderedList' and not p5condition and not lists):  # 98, dublication, 198
                                if (type(p1).__name__ == 'ResultSet'):
                                    path = p1[0]
                                else:
                                    path = p1
                                for unorderedlist in path.find_all('UnorderedList'):
                                    for listItem in unorderedlist.find_all('ListItem'):
                                        txt = txt + "  <p> " + listItem.find('Para').text + " </p>"
                                        for orderList in listItem.find_all('OrderedList'):
                                            for listItem1 in orderList.find_all('ListItem'):
                                                if (listItem1.has_key('NumberOverride')):
                                                    txt = txt + " <p> " + listItem1['NumberOverride']+ " </p>"
                                                txt = txt + "  <p> " + listItem1.find('Para').text+ " </p>"

                            elif (element.name == 'BlockText'):
                                for blockText in p1[0].select('BlockText > Para > Text'):
                                    txt = txt + blockText.text+ " </p>"

                if (first.select('P1para > P2') == None):
                    for P1para in first.select('P1para'):
                        txt = txt + " <p> " + P1para.find('Text').text+ " </p>"

                        # do we need this condition? # 270
                if (lists):
                    for unorderedlist in first.select('P1para > UnorderedList'):
                        for listItem in unorderedlist.find_all('ListItem'):
                            txt = txt + "  <p> " + listItem.find('Para').text+ " </p>"
                            for orderList in listItem.find_all('OrderedList'):
                                for listItem1 in orderList.find_all('ListItem'):
                                    if (listItem1.has_key('NumberOverride')):
                                        txt = txt + " <p> " + listItem1['NumberOverride']+ " </p>"
                                    txt = txt + "  <p> " + listItem1.find('Para').text+ " </p>"

                # last text - could require a for loop here # 99 case
                if (len(first.select('P1 > P1para > P2')) == 0):
                    x = len(first.find_all('Text')) - 1
                    txt = txt + " <p> " +first.find_all('Text')[x].text+ " </p>"

            return  Sections(titleSection, numberSection, parts)
           # return txt

        
def p3(block):
    return None

def p5(block):
    txt = " "
    txt = txt + block.select('Title')[0].text+" </p>"
    for p in block.select('P'):
        txt = txt + p.find('Text').text+" </p>"
        for unorderedlist in p.select('UnorderedList'):
            for listItem in unorderedlist.find_all('ListItem'):
                txt = txt + "  <p> " + listItem.find('Para').text+" </p>"
                for orderList in listItem.find_all('OrderedList'):
                    for listItem1 in orderList.find_all('ListItem'):
                        if (listItem1.has_key('NumberOverride')):
                            txt = txt + "  <p> " +listItem1['NumberOverride']+" </p>"
                        txt = txt + "  <p> " + listItem1.find('Para').text+" </p>"
        for p5 in p.select('P5'):
            txt = txt + p5.find('Pnumber').text+" </p>"
            for p5para in p5.select('P5para'):
                txt = txt + p5para.find('Text').text+" </p>"
    return txt

# no schedules captured
def exists (str, pattern):
    if pattern in str:
        print(str)

# report 77, 78

In [15]:
import re
import collections
    
def refInSingleLine(text):
    patterns=['see section','in section', 
              'of section', 'under section','within section',
               'with section','by section', 'that section',
              'omit section','also section','a section', 'to section',
              'and section', 'which section','amend section'
              'on section','than section','which section','the section','after section',
              'those sections','the section','before section','no section',
              'those section','or section','instead section','than section','despite section',
              'where section','for section', ' section ']
    
    if(not text.lower().startswith('sections ') and not text.lower().startswith('section ')):
        for y in patterns:
                if(y in text and not ( ("of the section" in text) or  ("c." in text)) ):
                    dependency=findDependency(text.lower(), y)
                    if(dependency is not None and len(dependency)>0):
                        return findDependency(text, y);
    else:                    
        if('Section' in text and (' to ' not in text)):
            return findMatch(text, 'Section')
        elif(text.startswith('section ')):
            return findMatch(text, 'section')
        elif(text.startswith('sections ')):
            text=text.replace("sections ","")
            if(" to " in text):
                return findToDependency(text);
            else:
                if ('of the' not in text):
                    return findMatch(text, 'sections')
                else:
                    return ''
        elif('Sections' in text and (' to ' not in text)):
            return findMatch(text, 'Sections')
        else:
            if((str(" to ") in text) and 'Parts' not in text):
                return findToDependency(text[0:text.index(' to ')+10]) #local text


def addPattern(array, text):
    if (isinstance(text, list)==True):
        for x in text:
            array.append(str(x))
        return array
    elif(text is not None):
        return array.append(str(text))
    else:
        return array

def addOutgoing(array, element):
    if(type(element)==re.Match):
        text=element.group(0);
        if('the' in text):
            text=text[text.rindex('the')+4:];
        if(text.rindex(' ') == len(text)-1): text=text[0:text.rindex(' ')];            
        return array.append(text);

def findMatch(text, pattern):
    text=subsectionBracketsRef(text)
    if(pattern in text):
        match=re.sub("[^0-9]", "", text[text.index(pattern)+len(pattern):])    
        if(match in text):
            numberOne=text[text.index(match):text.index(match)+len(match)]
            numberTwo=text[text.index(match):text.index(match)+len(match)+1]
            if(numberOne in numberTwo and len(numberOne)>0):
                match=numberTwo    
            return clearSection(match)
        else:
                # TODO consider later pattern occurrences - give details how many such cases we have. 
            match=re.sub("[^0-9][1]", "", text[text.index(pattern)+len(pattern)+1:])
            if(" " in match and " or " not in match and " to " not in match):
                return clearSection(match[0:match.index(" ")])
            else: 
                if(("ofthe" not in match) and "of the" not in text and ('Act' not in text)):
                    return clearSection(match)
    else:
        return text

def findDependency(text, pattern):    
    match=findMatch(text,pattern)
    if(clearSection(str(match)) in text and match!=None):
        if((str(" to "+pattern) in text)):
            return findToDependency(match)
        elif((str(" or "+pattern) in text)):
            return findOrDependency_1(text, pattern)
        elif((str(" and ") in text) and notSubesctions(text) and ofThe(text)):
            return findAndDependency(text, pattern)         
        elif(" of section" in text):
            text=text[text.index('of section')+11:]
            if(" " in text):
                text=text[0:text.index(' ')]
            return clearSection(text)        
        else:
            if(match != None):
                if(("of the" not in text)  and ('Act' not in text)):
                    return clearSection(str(match))
    else:
        if((str(" to ") in text)):
            return findToDependency(text[0:text.index(' to ')+10]) #local text
        elif((str(" followed ") in text)):
            return findDependency(text[0:text.index(' followed ')], pattern) 
        elif((str(" and ") in text)): 
            return findDependency(text[0:text.index(' and ')], pattern) 
        elif((str(" or ") in text) and (str(" or after") not in text)):
            return findOrDependency_1(text, pattern)
        else:
            if("(" in text and (pattern in text)):
                    match=re.sub("[^0-9]\(\d+\)", "", text[text.index(pattern)+len(pattern):])
                    if(")" in match):
                        return str(match[1:match.index(")")+1:])                        
            else:
                if(("ofthe" not in match) and "of the" not in text and ('Act' not in text)):
                    return str(match)
                            
def subsectionBracketsRef(text):
    if("(" in text):
        index_1=text.index("(")
        index_2=text.index(")")
        if(index_1<index_2):
            if( (index_1+2==index_2 or index_1+3==index_2) and notSubesctions(text)):
                return text[0:index_1]
            else:
                return text
        else:
             return text
    else:
        return text

def ofThe(text):
    if('of the' not in text):
        return True
    else:
        return False
    
def notSubesctions(text):
    if('subsection' not in text and 'Subsection' not in text):
        return True
    else:
        return False
    
def findToDependency(text):
    match=re.findall(r'\d+ to \d+', text)
    if(len(match)>0):
        text=re.findall(r'\d+ to \d+', text)[0]
        to=re.findall(r'\d+', text)
        return list(range(int(to[0]),int(to[1])+1))

def clearParentheses(text):
    return text[0:text.index("(")];

def subString(text, character):
    if(character in text):
        return text[0:text.index(character)];
    else:
        return text;
    
def removePrefix(text, character):
    return text[text.index(character):];

def removeSpace(text):
    return text.replace(" ","")

def clearSection(text):
    if(',' in text):
        text=subString(text, ',')
    if('“' in text):
        text=subString(text, '“')
    if('(' in text):
        text=subString(text, '(')
    if(')' in text):
        text=subString(text, ')')
    if(' ' in text):
        text=removeSpace(text)
    if('.' in text):
        text=subString(text, '.')    
    if(any(char.isdigit() for char in text)):
        return text.replace(";","")
    else:
        return ''

def keepNumbers(text):
    return  re.sub("[^0-9][1]", "", text)

def clearORText(text):
    match=re.sub("[a-zA-Z]", "", text)
    match=re.sub("\((\d+)\).", "", match)
    match=re.sub(" ", "", match)
    match=''.join(char for char in match if char.isalnum())
    return clearSection(match);

def findOrDependency(text, pattern):
    temp=text[text.index(pattern)+len(pattern)+1:text.index(' or ')]
    if("(" in temp):
        return [clearParentheses(temp), clearORText(text[text.index(' or ')+4:])]
    else: 
        return [temp, clearORText(text[text.index(' or ')+4:])]

def findOrDependency_1(text, pattern):
    match=re.sub("[^0-9] or [^0-9]", "", text[text.find(pattern)+len(pattern):])
    if(' or ' in match and ofThe(match)):
        numberTwo=match[match.index(' or ')+4:]
        if(' ' in numberTwo):
            numberTwo=numberTwo[0:numberTwo.index(' ')]
        else:
            numberTwo=clearSection(numberTwo)
        return [ match[0:match.index(' or ')], numberTwo]
    else:
        if(any(char.isdigit() for char in match) and ofThe(match) ):
            return clearSection(match)
        else:
            text=text[0:text.index(' or ')-2]
            if('section' in text):
                text=[text.index(' section ')+9]
            return text


def findAndDependency(text, pattern):
    match=re.sub("[^0-9] and [^0-9]", "", text[text.index(pattern)+len(pattern)+1:])
    if(' and ' in match):
        numberTwo=match[match.index(' and ')+5:]
        if('of that Act' not in match and 'of that Act' not in numberTwo):
            return [ clearSection(match[0:match.index(' and ')]), clearSection(numberTwo)]
    else:
        if('of that Act' not in text):
            return clearSection(text[text.index(pattern)+len(pattern)+1:])
def forSectionOfTheAct(text):
      match=re.sub("For Section [^0-9] of the", "", text)
      return clearSection(match)

In [16]:
def encode_subSections(obj):
    if isinstance(obj, Sections):
     return obj.__dict__
    if isinstance(obj, SuperSections):
     return obj.__dict__
    if isinstance(obj, subSections):
     return obj.__dict__
    if isinstance(obj, Paragraph):
     return obj.__dict__
    if isinstance(obj, subParagraph):
     return obj.__dict__
    if isinstance(obj, Line):
     return obj.__dict__
    return obj

## A repeated application of a regular pattern about outgoing Acts, within a line
def actsInSingleLine(line):
    line=line.replace("The", "the")
    try:
        found=re.search('the (.+?) Act (.+?) \(c. (.+?)\)', line[line.rfind("the"):])
        if(found==None):
            if(('the' in line) and ('Act' in line)):
                if(line.rindex('the')<line.index('Act')-30):
                    found=re.search('the (.+?) Act (\d\d\d\d) ', line[line.index("the")+1:])
                    if(found is None):
                        found=re.search('the (.+?) Act (\d\d\d\d)', line[line.rindex("the"):])
                else:
                    found=re.search('the (.+?) Act (\d\d\d\d) ', line[line.rindex("the"):])
                    if(found is None):
                        found=re.search('the (.+?) Act (\d\d\d\d)', line[line.rindex("the"):])
                        if(found is None):
                            found=re.search('the (.+?) Act (\d\d\d\d)', line[line.index("the"):])
            else:
                found=''
    except AttributeError:
        found=''
    if(found=='None'):
        return ''
    else:
        if(hasattr(found, 'group')):
            if(found.group(0)=='None'):
                return ''
            else:
                temp=found.group(0);
                return found;
        else:
            return found


        
def references(section, partNumber, partTitle, chapterNumber, chapterTitle, crossHeading):
    countSubsections=0;
    countParagraphs=0;
    countsubPargraphs=0;
    countLines=0;
    arraySubsections=[];
    references=[];
    outgoingReferences=[];
    for x in section.subSections:
        arrayParagraphs=[];
        arraysubParagraphs=[];
        addPattern(references, refInSingleLine(x.Title))
        addOutgoing(outgoingReferences, actsInSingleLine(x.Title))
        for y in x.Paragraphs:
            if(y.Title!=""):  countParagraphs=countParagraphs+1;
            addPattern(references, refInSingleLine(y.Title))
            addOutgoing(outgoingReferences, actsInSingleLine(y.Title))
            for z in y.subParagraphs:
                addPattern(references, refInSingleLine(z.Title))
                addOutgoing(outgoingReferences, actsInSingleLine(z.Title))
                countsubPargraphs=countsubPargraphs+1;
                arrayLines=[];
                for p in z.Lines:
                    if hasattr(p, 'Letter'):
                        arrayLines.append(Line(p.Letter , p.Title))
                        countLines=countLines+1;
                        addPattern(references, refInSingleLine(p.Title))
                        addOutgoing(outgoingReferences, actsInSingleLine(p.Title))
                if(z.Title!=None):
                    arraysubParagraphs.append(subParagraph(z.Number, z.Title, arrayLines));
            if(y.Number!=None):       
                arrayParagraphs.append(Paragraph(y.Number, y.Title, arraysubParagraphs));
        arraySubsections.append(subSections(x.Title, x.Number, arrayParagraphs));
        countSubsections=len(arraySubsections);
    addPattern(references, refInSingleLine(section.Title))
    addOutgoing(outgoingReferences, actsInSingleLine(section.Title))
    return SuperSections(Sections(section.Title, section.Number, arraySubsections),countSubsections,countParagraphs,countsubPargraphs,countLines,collections.Counter(references), collections.Counter(outgoingReferences), partNumber, partTitle, chapterNumber, chapterTitle, crossHeading);

In [18]:
import json


metadata={}
id='254';

arrayContents=[];

year=2004; number=34;

url="https://www.legislation.gov.uk/ukpga/2004/34/section/"+str(id)+"/data.xml";

url_1="https://www.legislation.gov.uk/ukpga/"+str(year)+'/'+str(number)+"/data.xml"

# Housing Act 2004, 34
parts=contents(url_1)

index=0;

countSubsections=0;
countParagraphs=0;
countsubPargraphs=0;
countLines=0;

for x in parts:
    print(str(x.Number).replace('\n',''))
    for y in x.Chapters:
        print(str(y.Number).replace('\n',''))
        for z in y.Sections:
            print(' '+str(z.Title).replace('\n',''))
            for t in z.SubSections:
                    print(str(t.Number)+" "+str(t.Title).replace('\n',''))
                    print(t.Url)
                    section=singlesection(t.Url)
                    superSection=references(section, str(x.Number).replace('\n',''), str(x.Title).replace('\n',''), str(y.Number).replace('\n',''), str(y.Title).replace('\n',''),' '+str(z.Title).replace('\n','') )
                    countSubsections=countSubsections+int(superSection.subSectionsNum);
                    countParagraphs=countParagraphs+int(superSection.paragraphsNum);
                    countsubPargraphs=countsubPargraphs+int(superSection.subParagraphsNum);
                    countLines=countLines+int(superSection.linesNum);
                    arrayContents.append(superSection)

metadata['sections']= arrayContents;
jsonFile = open("housingact2004.json", "w")
jsonFile.write(json.dumps(metadata, default=encode_subSections, indent=2))
jsonFile.close()

print("Sub Sections:"+str(countSubsections));
print("Paragraphs:"+str(countParagraphs));
print("Sub Paragraphs:"+str(countsubPargraphs));
print("Lines:"+str(countLines));


Part 1
Chapter 1
 New system for assessing housing conditions
1 New system for assessing housing conditions and enforcing housing standards
http://www.legislation.gov.uk/ukpga/2004/34/section/1/data.xml
2 Meaning of  “category 1 hazard” and  “category 2 hazard”
http://www.legislation.gov.uk/ukpga/2004/34/section/2/data.xml
 Procedure for assessing housing conditions
3 Local housing authorities to review housing conditions in their districts
http://www.legislation.gov.uk/ukpga/2004/34/section/3/data.xml
4 Inspections by local housing authorities to see whether category 1 or 2 hazards exist
http://www.legislation.gov.uk/ukpga/2004/34/section/4/data.xml
 Enforcement of housing standards
5 Category 1 hazards: general duty to take enforcement action
http://www.legislation.gov.uk/ukpga/2004/34/section/5/data.xml
6 Category 1 hazards: how duty under section 5 operates in certain cases
http://www.legislation.gov.uk/ukpga/2004/34/section/6/data.xml
7 Category 2 hazards: powers to take enforceme

 Grant or refusal of licences
 63 Applications for licences
http://www.legislation.gov.uk/ukpga/2004/34/section/63/data.xml
 64 Grant or refusal of licence
http://www.legislation.gov.uk/ukpga/2004/34/section/64/data.xml
 65 Tests as to suitability for multiple occupation
http://www.legislation.gov.uk/ukpga/2004/34/section/65/data.xml
 66 Tests for fitness etc. and satisfactory management arrangements
http://www.legislation.gov.uk/ukpga/2004/34/section/66/data.xml
 67 Licence conditions
http://www.legislation.gov.uk/ukpga/2004/34/section/67/data.xml
 68 Licences: general requirements and duration
http://www.legislation.gov.uk/ukpga/2004/34/section/68/data.xml
 Variation and revocation of licences
 69 Variation of licences
http://www.legislation.gov.uk/ukpga/2004/34/section/69/data.xml
 70 Power to revoke licences
http://www.legislation.gov.uk/ukpga/2004/34/section/70/data.xml
 70A Duty to revoke licence in banning order cases
http://www.legislation.gov.uk/ukpga/2004/34/section/70A/data.

127 Management orders: power to supply furniture
http://www.legislation.gov.uk/ukpga/2004/34/section/127/data.xml
128 Compensation payable to third parties
http://www.legislation.gov.uk/ukpga/2004/34/section/128/data.xml
129 Termination of management orders: financial arrangements
http://www.legislation.gov.uk/ukpga/2004/34/section/129/data.xml
130 Termination of management orders: leases, agreements and proceedings
http://www.legislation.gov.uk/ukpga/2004/34/section/130/data.xml
131 Management orders: power of entry to carry out work
http://www.legislation.gov.uk/ukpga/2004/34/section/131/data.xml
Chapter 2
 Introductory
132 Empty dwelling management orders: introductory
http://www.legislation.gov.uk/ukpga/2004/34/section/132/data.xml
 Interim empty dwelling management orders
133 Making of interim EDMOs
http://www.legislation.gov.uk/ukpga/2004/34/section/133/data.xml
134 Authorisation to make interim EDMOs
http://www.legislation.gov.uk/ukpga/2004/34/section/134/data.xml
135 Local hous

193 Right to buy: suspension of landlord’s obligation to complete
http://www.legislation.gov.uk/ukpga/2004/34/section/193/data.xml
194 Disclosure of information as to orders etc. in respect of anti-social behaviour
http://www.legislation.gov.uk/ukpga/2004/34/section/194/data.xml
Chapter 2
 Disposals by local authorities
195 Repayment of discount: periods and amounts applicable
http://www.legislation.gov.uk/ukpga/2004/34/section/195/data.xml
196 Repayment of discount: increase attributable to home improvements to be disregarded
http://www.legislation.gov.uk/ukpga/2004/34/section/196/data.xml
197 Local authority’s right of first refusal
http://www.legislation.gov.uk/ukpga/2004/34/section/197/data.xml
198 Deferred resale agreements
http://www.legislation.gov.uk/ukpga/2004/34/section/198/data.xml
 Disposals by registered social landlords
199 Repayment of discount: periods and amounts payable
http://www.legislation.gov.uk/ukpga/2004/34/section/199/data.xml
200 Registered social landlord’s r

 248 Timing and location of things done electronically
http://www.legislation.gov.uk/ukpga/2004/34/section/248/data.xml
 249 Proof of designations
http://www.legislation.gov.uk/ukpga/2004/34/section/249/data.xml
 Financial penalties as alternative to prosecution
 249A Financial penalties for certain housing offences in England
http://www.legislation.gov.uk/ukpga/2004/34/section/249A/data.xml
 Other supplementary provisions
 250 Orders and regulations
http://www.legislation.gov.uk/ukpga/2004/34/section/250/data.xml
 251 Offences by bodies corporate
http://www.legislation.gov.uk/ukpga/2004/34/section/251/data.xml
 252 Power to up-rate level of fines for certain offences
http://www.legislation.gov.uk/ukpga/2004/34/section/252/data.xml
 253 Local inquiries
http://www.legislation.gov.uk/ukpga/2004/34/section/253/data.xml
 Meaning of  “house in multiple occupation”
 254 Meaning of  “house in multiple occupation”
http://www.legislation.gov.uk/ukpga/2004/34/section/254/data.xml
 255 HMO declar